# 機械学習のためのSQL

## 問題設定

機械学習のためのSQLでは、式を暗記するより、計算の意味をコードで検証する姿勢を優先します。

前提: Pythonの基本文法と、平均・分散・相関の直感が前提です。

到達目標: 学習データの準備から評価指標の読み解きまで、モデル改善の往復を自力で回せる状態にします。

ここで扱う中心語は 「特徴量」、「目的変数」、「損失関数」、「汎化」、「過学習」、「機械学習のためのSQL」 です。用語を先に暗記するのではなく、コード実行の結果と結びつけて理解します。

このノートは 機械学習 分野の初学者向けに、説明とコードを交互に読み進める設計です。最初から完璧に理解する必要はありません。大切なのは、各コードの目的を一文で言えることと、出力が変わる理由を自分で確かめることです。

## 検証 1: SQLで集計する

機械学習の前段では SQL 集計が非常に重要です。メモリDBで実行し、前処理の入口を確認します。

In [ ]:
x = [1, 2, 3, 4, 5]
y = [3, 5, 7, 9, 11]
pairs = list(zip(x, y))
print('pairs =', pairs)
import sqlite3
con = sqlite3.connect(':memory:')
cur = con.cursor()
cur.execute('create table sales(day text, amount int)')
cur.executemany('insert into sales values (?, ?)', [('2026-01-01', 120), ('2026-01-02', 150), ('2026-01-03', 90)])
avg_amount = cur.execute('select avg(amount) from sales').fetchone()[0]
print('avg_amount =', round(avg_amount, 3))
con.close()

SQL で作る特徴量は再現しやすく、運用時のデータパイプラインにも直接つながります。

この節では、特徴量 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 検証 2: 単回帰を手で計算する

次に、最小二乗法の係数を手計算で求めます。既存ライブラリを使う前に中身を一度体験すると理解が安定します。

In [ ]:
x_bar = sum(x) / len(x)
y_bar = sum(y) / len(y)
num = sum((xi - x_bar) * (yi - y_bar) for xi, yi in pairs)
den = sum((xi - x_bar) ** 2 for xi in x)
w1 = num / den; w0 = y_bar - w1 * x_bar
print('w0, w1 =', round(w0, 4), round(w1, 4))

式の記号は抽象的に見えますが、コードでは平均・差分・和に分解されます。難しい式ほど、実装で部品に分けて確認するのが有効です。

この節では、特徴量 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 定義の確認

1. y_hat = f_theta(x)
2. L(theta) = (1/N) * sum_i l(y_hat_i, y_i)

## 検証 3: 予測と誤差を見る

ここで、求めた係数で予測を出し、誤差を数値化します。モデル改善は誤差の観察から始まります。

In [ ]:
pred = [w0 + w1 * xi for xi in x]
residual = [yi - pi for yi, pi in zip(y, pred)]
mse = sum((r ** 2) for r in residual) / len(residual)
print('pred =', pred)
print('mse =', round(mse, 8))

誤差を観察するときは、平均値だけでなく個別の残差も見てください。偏りがあると、モデル構造の見直しが必要になります。

この節では、特徴量 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 検証 4: 特徴量を拡張する

次に、特徴量を追加したときに表現力がどう変わるかを確認します。特徴量設計は精度に直接効く実装作業です。

In [ ]:
x2 = [xi ** 2 for xi in x]
feature = list(zip(x, x2))
print('feature sample =', feature[:3])
scaled_x = [(xi - x_bar) / (max(x) - min(x)) for xi in x]
print('scaled_x =', [round(v, 4) for v in scaled_x])

特徴量を増やすと表現力は上がりますが、同時に過学習リスクも増えます。増やす理由を説明できる特徴だけを採用する姿勢が重要です。

この節では、特徴量 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## 検証 5: 評価分割を体験する

最後に、学習と評価を分ける意味をコードで確かめます。分割しない評価は自己採点に近く、実運用の性能を過大評価しがちです。

In [ ]:
train_x, test_x = x[:3], x[3:]
train_y, test_y = y[:3], y[3:]
pred_test = [w0 + w1 * xi for xi in test_x]
mae_test = sum(abs(yi - pi) for yi, pi in zip(test_y, pred_test)) / len(test_y)
print('test mae =', round(mae_test, 6))

検証データでの誤差が急に悪化したら、モデルより先にデータ分布の差を疑ってください。この視点は実務で非常に重要です。

この節では、特徴量 が入出力のどこを決めるかを中心に読める状態になれば十分です。

## まとめ

今回のノートで押さえておくべき誤解しやすい点を整理します。

1. 訓練データと評価データの分布差を見ない
2. 単一スコアだけでモデルを選んでしまう
3. 前処理の漏れを評価後に気づく

機械学習のためのSQL はここまでです。最初のコードへ戻り、理解の変化を確認してください。